In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

import math
import numpy as np
import argparse
import glob
import cv2
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from __future__ import division
from math import cos, sin

In [2]:
image = cv2.imread("D:/coin.jpg", 1)

In [3]:
d = 1024 / image.shape[1]
dim = (1024, int(image.shape[0] * d))
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
cv2.imshow("input", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
output = image.copy()

In [5]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [6]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
gray = clahe.apply(gray)

In [7]:
def calcHistogram(img):
    # create mask
    m = np.zeros(img.shape[:2], dtype="uint8")
    (w, h) = (int(img.shape[1] / 2), int(img.shape[0] / 2))
    cv2.circle(m, (w, h), 60, 255, -1)
    h = cv2.calcHist([img], [0, 1, 2], m, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(h, h).flatten()

In [8]:
def calcHistFromFile(file):
    img = cv2.imread(file)
    return calcHistogram(img)

In [9]:
class Enum(tuple): __getattr__ = tuple.index

In [10]:
Material = Enum(('Copper', 'Brass', 'Euro1', 'Euro2'))

In [11]:
sample_images_copper = glob.glob("D:/Masterfile/sample_images/copper/*")
sample_images_brass = glob.glob("D:/Masterfile/sample_images/brass/*")
sample_images_euro1 = glob.glob("D:/Masterfile/sample_images/euro1/*")
sample_images_euro2 = glob.glob("D:/Masterfile/sample_images/euro2/*")

In [12]:
X = []
y = []
for i in sample_images_copper:
    X.append(calcHistFromFile(i))
    y.append(Material.Copper)
for i in sample_images_brass:
    X.append(calcHistFromFile(i))
    y.append(Material.Brass)
for i in sample_images_euro1:
    X.append(calcHistFromFile(i))
    y.append(Material.Euro1)
for i in sample_images_euro2:
    X.append(calcHistFromFile(i))
    y.append(Material.Euro2)

In [13]:
clf = MLPClassifier(solver="lbfgs")

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.4)

In [15]:
clf.fit(X_train, y_train)
score = int(clf.score(X_test, y_test) * 100)
print("Classifier mean accuracy: ", score)

Classifier mean accuracy:  97


In [16]:
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

In [17]:
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=2.2, minDist=100,
                           param1=200, param2=100, minRadius=50, maxRadius=120)

In [18]:

def predictMaterial(roi): 
    hist = calcHistogram(roi)
    s = clf.predict([hist])
    return Material[int(s)]



diameter = []
materials = []
coordinates = []

count = 0
if circles is not None:
    for (x, y, r) in circles[0, :]:
        diameter.append(r)

    circles = np.round(circles[0, :]).astype("int")
    for (x, y, d) in circles:
        count += 1
        coordinates.append((x, y))

        roi = image[y - d:y + d, x - d:x + d]
        material = predictMaterial(roi)
        materials.append(material)

        if False:
            m = np.zeros(roi.shape[:2], dtype="uint8")
            w = int(roi.shape[1] / 2)
            h = int(roi.shape[0] / 2)
            cv2.circle(m, (w, h), d, (255), -1)
            maskedCoin = cv2.bitwise_and(roi, roi, mask=m)
        

        cv2.circle(output, (x, y), d, (0, 255, 0), 2)
        cv2.putText(output, material,
                    (x - 40, y), cv2.FONT_HERSHEY_PLAIN,
                    1.5, (0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

In [19]:
d = 500 / output.shape[1]  
dim = (500, int(output.shape[0] * d))
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
output = cv2.resize(output, dim, interpolation=cv2.INTER_AREA)

In [20]:
cv2.imshow("Output",(output))
cv2.waitKey()
cv2.destroyAllWindows()
image2=cv2.imread("output.jpg",1)
print('done')

done


In [21]:
cv2.imwrite("D:/Coindetected.jpg", output) 


True

In [29]:
import cv2
import numpy as np

img = cv2.imread("D:/Coindetected.jpg", 1)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)

lines = cv2.HoughLines(edges,1,np.pi/180,200)
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imwrite("D:/houghlines3.jpg",img)

TypeError: 'NoneType' object is not subscriptable